In [2]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import time
import random

In [16]:
def encodestring(var):
    if var is not None:
        return var.encode('utf-8')

In [17]:
def scrapePainting(url):
    r = requests.get(url)
    print r, url
    b = BeautifulSoup(r.text, 'html.parser')
    
    url_id = r.url.split('/')[-1]
    
    artist = b.find('a', {'id':'fine-ART-ProductLabelArtistNameLink'})
    if artist is not None:
        artist = b.find('a', {'id':'fine-ART-ProductLabelArtistNameLink'}).text
        
    title = b.find('span', {'id': 'fineArtTitle'})
    if title is not None:
        title = b.find('span', {'id': 'fineArtTitle'}).text
        
    image = re.findall('http://ecx.images-amazon.com/images/I/.*\.jpg', r.text)
    if image is not None:
        image = re.findall('http://ecx.images-amazon.com/images/I/.*\.jpg', r.text)
        
    price = b.find('span', {'id': 'priceblock_ourprice'})
    if price is not None:
        price = b.find('span', {'id': 'priceblock_ourprice'}).text.replace('$','').replace(',', '')
        if '.' in price:
            price = float(price.replace('.', ''))/100
        else:
            price = float(price)
            
    desc = b.find('div', {'id': 'productDescription_feature_div'})
    if desc is not None:
        desc = b.find('div', {'id': 'productDescription_feature_div'}).getText().replace('\n','')
        
    height = None
    width = None
    size = b.find('span', {'id': 'mnba_buybox_size'})
    if size is not None:
        size = [a.strip() for a in b.find('span', {'id': 'mnba_buybox_size'}).text.split('x')]
        if len(size) == 2:
            height = float(size[0].replace('in.','').strip())
            width = float(size[1].replace('in.','').strip())
            depth = 1.
        elif len(size) == 3:
            height = float(size[0].replace('in.','').strip())
            width = float(size[1].replace('in.','').strip())
            depth = float(size[2].replace('in.','').strip())
    
    size_variations = b.find('div', {'id': 'variation_size_name'})
    if size_variations is not None:
        size = [a.strip() for a in size_variations.find('span', {'class': "a-size-base"}).text.split('x')]
        
        if len(size) == 2:
            height = float(size[0].replace('in.','').strip())
            width = float(size[1].replace('in.','').strip())
            depth = 1.
        elif len(size) == 3:
            height = float(size[0].replace('in.','').strip())
            width = float(size[1].replace('in.','').strip())
            depth = float(size[2].replace('in.','').strip())
            
        price = size_variations.find('span', {'class': "a-size-mini"}).text.replace('$','').replace(',', '')
        if '.' in price:
            price = float(price.replace('.', ''))/100
        else:
            price = float(price)
    

    return {'url_id': encodestring(url_id), 'artist': encodestring(artist), 'title':encodestring(title), 'image': image, 'price': price, 
            'description':encodestring(desc), 'height': height, 'width': width, 'size':size}

In [18]:
scrapePainting('http://www.amazon.com/17x22-in-Harold-Feinstein-Magnolia/dp/B00E5MTRB2')

<Response [200]> http://www.amazon.com/17x22-in-Harold-Feinstein-Magnolia/dp/B00E5MTRB2


{'artist': 'Harold Feinstein',
 'description': '"I first began shooting flowers in the mid 80?s. I would go up to the roof of my small Greenwich village studio and hold blossoms up against the sky to see the translucence of the petals. That series of 35mm photographs was entitled Sky Flowers and I had many of them printed as dye transfers and cibachromes. In the late 90?s I began to experiment with digital photography and became one of the first to use a scanner as a camera, which resulted in seven books of large format color photographs. Five of them were of flowers, foliage and plant life. When asked about my technique, I simply reply, "A prayer of gratitude - for the beauty of the flowers, and the eyes to see them with." Prints have a minimum of 0.5" border on all sides. Generally the color prints are centered on the paper. (ie for paper size 13"x19" the image size would be 11"x11").',
 'height': 13.0,
 'image': [u'http://ecx.images-amazon.com/images/I/51CuD4iaMtL._QL70_.jpg',
  u'h

In [ ]:
for page in range(1,4):
    
    page_url = "http://www.amazon.com/s/ref=sr_pg_3?rh=n%3A4991425011%2Cn%3A%214991426011%2Cn%3A6685269011%2Cn%3A6685289011&page="+str(page)+"&ie=UTF8&qid=1462467907"
    r = requests.get(page_url)
    r.text

    b = BeautifulSoup(r.text, 'html.parser')

    url_1 = []
    for link in b.findAll('a', {'class':"a-link-normal a-text-normal"}):
        url_1.append(link.get('href'))
    
    url_list = url_1[::2]

    paintings = []
    for url in url_list:
        try:
            paintings.append(scrapePainting(url))
        except Exception as error:
            print error
        time.sleep(random.randint(5,20))

df = pd.DataFrame(paintings)

In [ ]:
df

In [ ]:
df.to_csv("paintings_first_df.csv", index=False)